<a href="https://colab.research.google.com/github/cristiandarioortegayubro/UNI/blob/main/fe_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](https://github.com/cristiandarioortegayubro/UNI/blob/main/UNI%2039%20Aniversario.jpg?raw=true)

# **Fundamentos de Estadística**


## **Analisis de Normalidad**

El análisis de normalidad tiene como objetivo comprobar si los datos disponibles podrían proceder de una población con una distribución normal.

Existen tres estrategias principales para abordar este análisis:

+ Representaciones gráficas

+ Métodos analíticos

+ Tests

## **Librerías**

### **Para análisis estadístico**

In [1]:
# Operaciones matemáticas y estadísticas
import pandas as pd
import numpy as np
import scipy.stats as stats

### **Para gráficos**

In [38]:
# Visualización
import plotly.express as px
import plotly.graph_objs as go

## **Conjunto de Datos**

Los datos utilizados en este ejemplo se han obtenido del libro Statistical Rethinking by Richard McElreath. El set de datos contiene información recogida por Nancy Howell a finales de la década de 1960 sobre el pueblo !Kung San, que viven en el desierto de Kalahari entre Botsuana, Namibia y Angola.

In [3]:
url = ('https://raw.githubusercontent.com/cristiandarioortegayubro/UNI/main/Howell1.csv')
datos = pd.read_csv(url)
datos.head()

,height,weight,age,male
0,151.765,47.825606,63.0,1
1,139.700,36.485807,63.0,0
2,136.525,31.864838,65.0,0
3,156.845,53.041914,41.0,1
4,145.415,41.276872,51.0,0


In [4]:
datos.shape

(544, 4)

In [5]:
datos = datos[(datos.age > 15) & (datos.male ==0)]

## **Representación gráfica**

***Uno de los métodos gráficos más empleados para el análisis de normalidad consiste en representar los datos mediante un histograma y superponer la curva de una distribución normal con la misma media y desviación estándar que los datos disponibles.***

Valores de la media (mu) y desviación estándar (sigma) de los datos del peso (weight)

In [6]:
mu, sigma = stats.norm.fit(datos.weight)

In [7]:
mu

41.38615752272728

In [8]:
sigma

5.737813252284781

Valores teóricos de la normal en el rango observado

In [9]:
datos.weight.min()

22.6796

In [10]:
datos.weight.max()

56.755699

In [11]:
x = np.linspace(datos.weight.min(),
                datos.weight.max(),
                num = 100)

In [12]:
y = stats.norm.pdf(x, mu, sigma)

In [37]:
fig = go.Figure([go.Scatter(x = x,
                            y = y,
                            line = {"width":2},
                            name = "Función densidad de probabilidad"),
                 
                 go.Histogram(x = x,
                              y = y,
                              nbinsx = 18,
                              histfunc = "avg",
                              histnorm = "probability density",
                              showlegend = False,
                              name = "Peso")])

fig.update_layout(template =    "simple_white",
                  title =       "Distribución peso mujeres mayores de 15 años",
                  yaxis_title = "Función densidad de probabilidad",
                  xaxis_title = "Peso")

fig.show()

***Otra representación utilizada con frecuencia son los gráficos de cuantiles teóricos (Gráficos Q-Q). Estos gráficos comparan los cuantiles de la distribución observada con los cuantiles teóricos de una distribución normal con la misma media y desviación estándar que los datos. Cuanto más se aproximen los datos a una normal, más alineados están los puntos entorno a la recta.***

In [14]:
from statsmodels.graphics.gofplots import qqplot

In [ ]:
qqplot_data = qqplot(data=y, line='s').gca().lines

In [36]:
fig = go.Figure([go.Scatter(x = qqplot_data[0].get_xdata(),
                            y = qqplot_data[0].get_ydata(),
                            mode = "markers",
                            name = "Cuantiles de la distribución"),
                 
                 go.Scatter(x = qqplot_data[1].get_xdata(),
                            y = qqplot_data[1].get_ydata(),
                            line = {"width":2},
                            name = "Cuantiles teoricos")])

fig.update_layout(template =    "simple_white",
                  title =       "Q-Q Plot",
                  yaxis_title = "Cuantiles de la distribución",
                  xaxis_title = "Cuantiles teoricos")

fig.show()

## **Métodos analíticos asimetría y curtosis**

Los estadísticos de asimetría (Skewness) y curtosis pueden emplearse para detectar desviaciones de la normalidad. Un valor de curtosis y/o coeficiente de asimetría entre -1 y 1, es generalmente considerada una ligera desviación de la normalidad (Bulmer, 1979), (Brown, n.d.). Entre -2 y 2 hay una evidente desviación de la normal pero no extrema

### **Asimetría**

In [41]:
asimetria = stats.skew(datos.weight)
print("")
print(f"La asimetría es {round(asimetria, 3)}")


La asimetría es 0.032


### **Curtosis**

In [42]:
curtosis = stats.kurtosis(datos.weight)
print("")
print(f"La curtosis es {round(curtosis, 3)}")


La curtosis es 0.055


## **Tests**

Los test Shapiro-Wilk test y D'Agostino's K-squared test son dos de los test de hipótesis más empleados para analizar la normalidad. En ambos, se considera como hipótesis nula que los datos proceden de una distribución normal.

El p-value de estos test indica la probabilidad de obtener unos datos como los observados si realmente procediesen de una población con una distribución normal con la misma media y desviación que estos. Por lo tanto, si el p-value es menor que un determinado valor (típicamente 0.05), entonces se considera que hay evidencias suficientes para rechazar la normalidad.

El test de Shapiro-Wilk se desaconseja cuando se dispone de muchos datos (más de 50) por su elevada sensibilidad a pequeñas desviaciones de la norma

### **Shapiro-Wilk**

In [45]:
shapiro_test = stats.shapiro(datos.weight)
shapiro_test

ShapiroResult(statistic=0.9963726997375488, pvalue=0.9239704012870789)

In [46]:
estadistico = shapiro_test[0]
pvalor = shapiro_test[1]

In [51]:
print(f"Estadístico = {round(estadistico, 4)} \np-value = {round(pvalor,4)}")

Estadístico = 0.9964 
p-value = 0.924


### **D'Agostino's K-squared**

In [54]:
k2, p_value = stats.normaltest(datos.weight)

In [55]:
print(f"Estadístico = {round(k2, 4)} \np-value = {round(p_value, 4)}")

Estadístico = 0.199 
p-value = 0.9053


***Ninguno de los test muestra evidencias para rechazar la hipótesis de que los datos se distribuyen de forma normal (p-value muy próximo a 1)***